In [1]:
%matplotlib inline

import warnings
import time
import sys
import numpy as np
import matplotlib.pyplot as plt
from hmmlearn import hmm

import random
import pandas as pd

pd.options.mode.use_inf_as_na = True
# warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
NUM_ITERS=10000
DEFAULT_TOL = 1e-12

In [3]:
likelihood_vect = np.empty([0,1])
aic_vect = np.empty([0,1])
bic_vect = np.empty([0,1])

# Possible number of states in Markov Model
STATE_SPACE = range(2,15)

In [22]:
stock = "BABA"
# dataset = np.genfromtxt('../Dataset/' + stock + '.csv', delimiter=',')
csv_data = pd.read_csv('../Dataset/' + stock + '.csv')

In [23]:
# clean up data
# delete extra data (Date[0],Adj Close[4], Volume[5]) and leave Open/Hight/Low/Close
del(csv_data['Date'])
del(csv_data['Adj Close'])
del(csv_data['Volume'])

csv_data.head()

,Open,High,Low,Close
0,92.699997,99.699997,89.949997,93.889999
1,92.699997,92.949997,89.500000,89.889999
2,88.940002,90.480003,86.620003,87.169998
3,88.470001,90.570000,87.220001,90.570000
4,91.089996,91.500000,88.500000,88.919998


In [24]:
# csv_data['Volume'] = (csv_data['Volume'] / 1000).astype(int) / 100
# csv_data.head()

In [25]:
def get_exp_preprocessing(df,alpha=0.9):
    edata = df.ewm(alpha=alpha).mean()
    return edata

In [26]:
dataset = get_exp_preprocessing(csv_data)
dataset.head()

,Open,High,Low,Close
0,92.699997,99.699997,89.949997,93.889999
1,92.699997,93.563633,89.540909,90.253635
2,89.312614,90.785588,86.909462,87.475584
3,88.554186,90.591539,87.188975,90.260837
4,90.836438,91.409162,88.368909,89.054070


In [9]:
# dataset = (dataset*100).astype(int)/100
# dataset[:,-1] = dataset[:,-1] / 1000

In [27]:
predicted_stock_data = np.empty([0,dataset.shape[1]])
likelihood_vect = np.empty([0,1])
aic_vect = np.empty([0,1])
bic_vect = np.empty([0,1])

In [28]:
dataset = np.array(dataset)

In [29]:
opt_states = 10
K = 50

In [30]:
model = hmm.GaussianHMM(n_components=opt_states, covariance_type='full', tol=DEFAULT_TOL, n_iter=NUM_ITERS, init_params='stmc')
model.fit(dataset)

curr_likelihood = model.score(dataset[-K:])

In [42]:
all_pass_likelihood = []
for idx in range(dataset.shape[0]-K-1):
    all_pass_likelihood = np.append(all_pass_likelihood, model.score(dataset[idx:idx+K]))

In [45]:
# most last likelihood
likelihood_diff_idx = np.argmin(np.absolute(all_pass_likelihood - curr_likelihood))
likelihood_diff_idx

846

In [93]:
# take first K items as init history data
# test_pass_likelihood = all_pass_likelihood[K:-3]

currect_rate = 0
for i in range(K,all_pass_likelihood.shape[0]-K-3):
    actual_idx = i+K+1
    
    item_test_pass_likelihood = all_pass_likelihood[:actual_idx]
    item_test_cur_likelihood = all_pass_likelihood[actual_idx+1]
    idx = K + np.argmin(np.absolute(item_test_pass_likelihood - item_test_cur_likelihood))
    
    # predict the price and percentage
    diff_price = dataset[idx+1][3] - dataset[idx][3]
    diff_price_percent = (dataset[idx+1][3] / dataset[idx][3]) - 1.0
    
    # use actual price to count the accuracy
    actual_diff_price = dataset[actual_idx+1][3] - dataset[actual_idx][3]
    actual_diff_percent = (dataset[actual_idx+1][3] / dataset[actual_idx][3]) - 1.0
    
    if (diff_price > 0 and actual_diff_price > 0) or (diff_price < 0 and actual_diff_price < 0):
        currect_rate += 1

In [94]:
# print(all_pass_likelihood.shape)
# print(item_test_pass_likelihood.shape)
# print(test_pass_likelihood.shape)
# print(all_pass_likelihood[:K+i+1])
# print(item_test_cur_likelihood)
print("准确率", currect_rate / (all_pass_likelihood.shape[0]-(K*2)-3))

准确率 0.5067720090293454
